In [ ]:
import pyspark

In [2]:
pyspark.__version__

'3.4.3'

In [11]:
from pyspark.sql import SparkSession

In [51]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [29]:
df = spark.read.csv('fhvhv_tripdata_2021-01.csv')

In [30]:
df.schema

StructType([StructField('_c0', StringType(), True), StructField('_c1', StringType(), True), StructField('_c2', StringType(), True), StructField('_c3', StringType(), True), StructField('_c4', StringType(), True), StructField('_c5', StringType(), True), StructField('_c6', StringType(), True)])

In [4]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [6]:
import pandas as pd

In [7]:
df_pandas = pd.read_csv('head.csv')

In [8]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [13]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [14]:
from pyspark.sql import types

In [15]:
schema = types.StructType([    
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True)
])

In [33]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("fhvhv_tripdata_2021-01.csv")

In [32]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [34]:
df = df.repartition(24)

In [35]:
df.write.parquet('fhvhv/2021/01/')

24/08/22 19:51:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/08/22 19:51:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/08/22 19:51:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/08/22 19:51:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
24/08/22 19:51:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
24/08/22 19:51:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
24/08/22 19:51:07 WARN MemoryManager: Total allocation exceeds 95.

In [56]:
df = spark.read.parquet('fhvhv/2021/01/')

In [58]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [52]:
from pyspark.sql import functions as F

In [64]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [65]:
crazy_stuff('B02884')

's/b44'

In [62]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date',F.to_date(df.dropoff_datetime)) \
    .select('pickup_date','dropoff_date','PULocationID','DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-02|  2021-01-02|         173|          92|
| 2021-01-01|  2021-01-01|          37|         225|
| 2021-01-02|  2021-01-02|         220|         153|
| 2021-01-01|  2021-01-01|         232|         137|
| 2021-01-02|  2021-01-02|         188|          71|
| 2021-01-02|  2021-01-02|          74|         140|
| 2021-01-01|  2021-01-01|          35|          39|
| 2021-01-01|  2021-01-01|         191|         244|
| 2021-01-01|  2021-01-01|         216|         216|
| 2021-01-01|  2021-01-01|         191|         130|
| 2021-01-01|  2021-01-01|          78|          51|
| 2021-01-02|  2021-01-02|         127|         243|
| 2021-01-02|  2021-01-02|         102|         198|
| 2021-01-01|  2021-01-01|         100|         246|
| 2021-01-02|  2021-01-02|          76|         177|
| 2021-01-02|  2021-01-03|         138|       

In [59]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
.filter(df.hvfhs_license_num == 'HV0003') \
.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 01:07:28|2021-01-01 01:18:22|         232|         137|
|2021-01-02 02:32:21|2021-01-02 02:44:52|         188|          71|
|2021-01-01 14:11:55|2021-01-01 14:23:01|          35|          39|
|2021-01-01 23:32:43|2021-01-01 23:39:46|         216|         216|
|2021-01-01 22:22:35|2021-01-01 22:37:53|         191|         130|
|2021-01-01 15:24:34|2021-01-01 15:34:51|          78|          51|
|2021-01-02 09:12:07|2021-01-02 09:20:28|         127|         243|
|2021-01-02 14:26:19|2021-01-02 14:34:21|         102|         198|
|2021-01-02 16:25:57|2021-01-02 16:36:11|         113|         231|
|2021-01-02 21:11:44|2021-01-02 21:48:10|         211|         265|
|2021-01-01 19:26:51|2021-01-01 19:56:42|         259|         216|
|2021-01-01 17:52:59|2021-01-01 18:12:48|       